In [10]:
import pandas as pd
from numpy.f2py.auxfuncs import throw_error

### Création de l'authentification API Spotify

In [1]:
import os

if os.path.exists("./spotify_cache"):
    os.remove("./spotify_cache")
    print("Custom cache file deleted.")
else:
    print("Custom cache file not found.")

Custom cache file deleted.


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

SPOTIFY_CLIENT_ID = "ed9ece4a286d42f1aa834c82bed1f95e"
SPOTIFY_CLIENT_SECRET = "e1f250bb36914f3c9235c88df325eff6"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri="http://localhost:8000/callback",
    scope="user-top-read playlist-read-private user-library-read playlist-modify-public playlist-modify-private",
    cache_path="./spotify_cache",
    state="unique_state_value"
))

### Get the user's top tracks

In [6]:
# Prompt the user to enter the playlist name
playlist_name = input("Enter the playlist name: ")

# Search for the playlist by name
results = sp.search(q=f'playlist:{playlist_name}', type='playlist')

# Check if any playlists were found
if not results['playlists']['items']:
    print(f"Playlist named '{playlist_name}' not found.")
else:
    # Get the first playlist's ID
    playlist_id = results['playlists']['items'][0]['id']

    # Get all tracks from the playlist
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']

    # Extract track IDs, names, and authors
    track_info = [
        (track['track']['id'], track['track']['name'], track['track']['artists'][0]['name'])
        for track in tracks if track['track'] is not None
    ]

    # Print track IDs, names, and authors
    for track_id, track_name, track_author in track_info:
        print(f"Track ID: {track_id}, Track Name: {track_name}, Author: {track_author}")

Track ID: 01Lr5YepbgjXAWR9iOEyH1, Track Name: Love Sosa, Author: Chief Keef
Track ID: 6gBFPUFcJLzWGx4lenP6h2, Track Name: goosebumps, Author: Travis Scott
Track ID: 4Li2WHPkuyCdtmokzW2007, Track Name: Ni**as In Paris, Author: JAY-Z
Track ID: 7GX5flRQZVHRAGd6B4TmDO, Track Name: XO Tour Llif3, Author: Lil Uzi Vert
Track ID: 7KXjTSCq5nL1LoYtL7XAwS, Track Name: HUMBLE., Author: Kendrick Lamar
Track ID: 1bDbXMyjaUIooNwFE9wn0N, Track Name: Rich Flex, Author: Drake
Track ID: 1e1JKLEDKP7hEQzJfNAgPl, Track Name: Magnolia, Author: Playboi Carti
Track ID: 7ycWLEP1GsNjVvcjawXz3z, Track Name: Praise The Lord (Da Shine) (feat. Skepta), Author: A$AP Rocky
Track ID: 5ECewKkACFqVIxIDadOwnj, Track Name: Mind Over Matter, Author: StoneyIsland
Track ID: 119c93MHjrDLJTApCVGpvx, Track Name: C.R.E.A.M. (Cash Rules Everything Around Me), Author: Wu-Tang Clan
Track ID: 2iHNfqJjQRapX0qL25Sy6f, Track Name: bold claims, Author: carnivxre
Track ID: 2U5WueTLIK5WJLD7mvDODv, Track Name: Wishing Well, Author: Juice WR

### Get the track IDs to fetch audio features

In [8]:
track_ids = [track[0] for track in track_info]

print(f"Track IDs: {track_ids}")

Track IDs: ['01Lr5YepbgjXAWR9iOEyH1', '6gBFPUFcJLzWGx4lenP6h2', '4Li2WHPkuyCdtmokzW2007', '7GX5flRQZVHRAGd6B4TmDO', '7KXjTSCq5nL1LoYtL7XAwS', '1bDbXMyjaUIooNwFE9wn0N', '1e1JKLEDKP7hEQzJfNAgPl', '7ycWLEP1GsNjVvcjawXz3z', '5ECewKkACFqVIxIDadOwnj', '119c93MHjrDLJTApCVGpvx', '2iHNfqJjQRapX0qL25Sy6f', '2U5WueTLIK5WJLD7mvDODv', '51Fjme0JiitpyXKuyQiCDo', '03lR5LGBPWsdv9kQQ9mJPu', '4tqcoej1zPvwePZCzuAjJd', '6f4aMrTLti8gIzb0b5VZ02', '4PYfFHKfjYz6dwAtZhBy1z', '4VQNCzfZ3MdHEwwErNXpBo', '7ggbuLMTKpeisrRZw90NK2', '3gY6tiCNsuVi6s8kPV6aQg', '0zeA5Q5gWRvquhEBO58ldv', '0PV1TFUMTBrDETzW6KQulB', '53BZ6XygAoubR5DU5w38Vq', '6kbOxzbmDpDoif6qfNlsJf', '3ia3dJETSOllPsv3LJkE35', '1cOcVjghnmA0TvfM91Xrjy', '6MdqqkQ8sSC0WB4i8PyRuQ', '3QFInJAm9eyaho5vBzxInN', '3WxBSE7ycdD7g39mK1z8tA', '0XgpiStoxq1IJncYlPrvZ5', '4NU4xuddcBSoaKF33acIbq', '3Y6XWs8xMlCngyIxNOFnsp', '6gI9b2VsoWhjhIuIeToDVs', '6HgGuHx0FaeieNZhsY4g5y', '7sO5G9EABYOXQKNPNiE9NR', '4wZSM5C84LBLKkCxZNqeP4', '0judK8TqYfvwUohBngUxrE', '0FA4wrjDJvJTTU8AepZTup', 

### Load data_appended dataset and fetch the top tracks using ids

In [12]:
data = pd.read_csv("data/data_appended.csv")

top_tracks_data = data[data['id'].isin(track_ids)]

print("Top tracks data :")
print("Found a total of {} tracks".format(len(top_tracks_data)))
print(top_tracks_data)

track_to_analyse = top_tracks_data['id'].tolist()

print(track_to_analyse)

Top tracks data :
Found a total of 36 tracks
        valence  year  acousticness  \
14210    0.5760  1993      0.570000   
14234    0.6880  1993      0.125000   
14351    0.3240  1993      0.124000   
14414    0.7770  1994      0.472000   
14649    0.2410  1995      0.125000   
14669    0.8210  1995      0.028700   
14813    0.6700  1996      0.303000   
14904    0.4000  1996      0.137000   
15005    0.7460  1996      0.210000   
15239    0.7400  1998      0.056400   
15248    0.7610  1998      0.030400   
15332    0.8140  1998      0.034700   
15542    0.9290  1999      0.006020   
15568    0.8390  1999      0.045400   
17071    0.7750  2007      0.016400   
17145    0.9120  2007      0.398000   
17816    0.7750  2011      0.127000   
18459    0.1800  2014      0.050500   
19010    0.4010  2017      0.002640   
19012    0.3330  2017      0.149000   
19018    0.4210  2017      0.000282   
19032    0.3490  2017      0.259000   
19100    0.4430  2017      0.011400   
19233    0.2250  20

In [40]:
#Throw an error to stop the execution here
throw_error()

TypeError: throw_error.__init__() missing 1 required positional argument: 'mess'

In [13]:
from useKmean import FindSimilarSong

f = FindSimilarSong()
result = f.find_songs_for_playlist(track_to_analyse)




### Create Spotify playlist

In [15]:
# Get user ID (required to create a playlist)
user_id = sp.current_user()['id']

# Create a new playlist
playlist_name = "AI-ify Playlist"
playlist_description = "This is a playlist created using AI-ify."
playlist = sp.user_playlist_create(user_id, playlist_name, public=True, description=playlist_description)

# Add tracks to the playlist
sp.playlist_add_items(playlist['id'], result)

{'snapshot_id': 'AAAAAjfKI5m2AWetmuPH8SAiPkBZ1f66'}